In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/My Drive/Independent Study/Code/data'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import os
import pickle
from tensorflow.keras import Model
tf.keras.backend.set_floatx('float64')

In [ ]:
# with open("data.unpaired_clean_augment.pkl", 'rb') as f:
#   train_data = pickle.load(f)

In [ ]:
# t,x,y,status,evt=[],[],[],[],[]
# for i in train_data:
#   for data in i:
#     # t.append(data[0])
#     x.append(data[1])
#     y.append(data[2])
#     status.append(data[3])
#     evt.append(data[4])

In [ ]:
# df_train = pd.DataFrame(list(zip(x,y,status,evt)),columns =['x', 'y', 'status', 'evt'])
# df_train["status"] = df_train["status"].astype(float)

In [ ]:
df_X_Train = pd.read_csv("LundX.csv")
df_Y_Train = pd.read_csv("LundY.csv")

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
time_steps = 100
X_train, y_train = create_dataset(df_X_Train, df_Y_Train, time_steps)

In [ ]:
def gaussian2d(x1, x2, mu1, mu2, s1, s2, rho):
    # define gaussian mdn (eq 24, 25 from http://arxiv.org/abs/1308.0850)
    x_mu1 = tf.subtract(x1, mu1)
    x_mu2 = tf.subtract(x2, mu2)
    Z = tf.square(tf.compat.v1.div(x_mu1, s1)) + \
        tf.square(tf.compat.v1.div(x_mu2, s2)) - \
        2*tf.compat.v1.div(tf.multiply(rho, tf.multiply(x_mu1, x_mu2)), tf.multiply(s1, s2))
    rho_square_term = 1-tf.square(rho)
    power_e = tf.exp(tf.compat.v1.div(-Z,2*rho_square_term))
    regularize_term = 2*np.pi*tf.multiply(tf.multiply(s1, s2), tf.sqrt(rho_square_term))
    gaussian = tf.compat.v1.div(power_e, regularize_term)
    return gaussian

def get_loss(pi, x1_data, x2_data, eos_data, evt_data, mu1, mu2, sigma1, sigma2, rho, eos, evt):
# define loss function (eq 26 of http://arxiv.org/abs/1308.0850)
  gaussian = gaussian2d(x1_data, x2_data, mu1, mu2, sigma1, sigma2, rho)
  term1 = tf.multiply(gaussian, pi)
  term1 = tf.reduce_sum(input_tensor=term1, axis=1, keepdims=True) #do inner summation
  term1 = -tf.math.log(tf.maximum(term1, 1e-20)) # some errors are zero -> numerical errors.

  term2 = tf.multiply(eos, eos_data) + tf.multiply(1-eos, 1-eos_data) #modified Bernoulli -> eos probability
  term2 = -tf.math.log(tf.maximum(term2, 1e-20)) #negative log error gives loss

  term3 = tf.nn.sigmoid_cross_entropy_with_logits(evt, evt_data, name=None)

  return term1, term2, term3

  #transform dense NN outputs into params for MDN
def get_mdn_coef(Z):
  # returns the tf slices containing mdn dist params (eq 18...23 of http://arxiv.org/abs/1308.0850)
  eos_hat = Z[:, 0:1] #end of event tokens
  evt_hat = Z[:, 1:3+1] #evt

  pi_hat, mu1_hat, mu2_hat, sigma1_hat, sigma2_hat, rho_hat = tf.split(Z[:, 3+1:], 6, 1)
  pi_hat, sigma1_hat, sigma2_hat = \
                              pi_hat, sigma1_hat, sigma2_hat # these are useful for bias method during sampling

  eos = tf.sigmoid(1*eos_hat)
  pi = tf.nn.softmax(pi_hat) # softmax z_pi:
  mu1 = mu1_hat; mu2 = mu2_hat # leave mu1, mu2 as they are
  sigma1 = tf.exp(sigma1_hat); sigma2 = tf.exp(sigma2_hat) # exp for sigmas
  rho = tf.tanh(rho_hat) # tanh for rho (squish between -1 and 1)

  return [eos, evt_hat, pi, mu1, mu2, sigma1, sigma2, rho]

In [ ]:
class mdn(keras.layers.Layer):
    def __init__(self,rnn_size=128,n_out=124,**kwargs):
        super(mdn, self).__init__()
        # n_out = 3 + 1 + 20 * 6
        graves_initializer = tf.initializers.TruncatedNormal(mean=0.,stddev=.075,seed=None)
        self.mdn_w = tf.Variable(initial_value = graves_initializer(shape=[rnn_size, n_out], dtype=tf.float64),name="output_w",trainable=True)
        self.mdn_b = tf.Variable(initial_value = graves_initializer(shape=[n_out], dtype=tf.float64),name="output_b",trainable=True)
      

    def call(self, outputs):
      output = tf.reshape(outputs, [-1, 128])
      output = tf.compat.v1.nn.xw_plus_b(output, self.mdn_w, self.mdn_b)
      return output

    def get_config(self):

      config = super().get_config().copy()
      return config

In [ ]:
def get_final_loss(y_true, y_pred):
   
    x1_data = tf.gather(y_true,[0],axis=1)
    x2_data = tf.gather(y_true,[1],axis=1)
    eos_data = tf.gather(y_true,[2],axis=1)
    evt_data = tf.gather(y_true,[3,4,5],axis=1)
    [eos, evt,pi, mu1, mu2, sigma1,sigma2, rho] = get_mdn_coef(y_pred)

    losses = get_loss(pi, x1_data, x2_data, eos_data, evt_data, \
                            mu1, mu2, sigma1, sigma2, rho, \
                            eos, evt)
    
    loss = tf.reduce_sum(sum(losses))
    cost = loss / (50 * 100)
    return cost 

In [ ]:
model = keras.Sequential()
model.add(tf.keras.layers.LSTM(128,dropout=0.25,return_sequences=True))
model.add(tf.keras.layers.LSTM(128,dropout=0.25,return_sequences=True))
model.add(tf.keras.layers.LSTM(128,dropout=0.25))
model.add(mdn())
model.compile(optimizer=tf.keras.optimizers.RMSprop(clipnorm=10.0,learning_rate = 0.001,momentum=0.9,decay=0.9),loss=get_final_loss)

In [ ]:
epochs = 100000
step_ = 2000
batch_size = 50

In [ ]:
optimizer=tf.keras.optimizers.RMSprop(clipnorm=10.0,learning_rate = 0.001,momentum=0.9,decay=0.9)

In [ ]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().batch(batch_size)

In [ ]:
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restored from {}".format(manager.latest_checkpoint))
else:
  print("Initializing from scratch.")

epochs_done = manager.checkpoint.save_counter.numpy()
for epoch in range(epochs_done,epochs):
  avg_loss = []
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

      with tf.GradientTape() as tape:
          logits = model(x_batch_train, training=True)  
          loss_value = get_final_loss(y_batch_train, logits)

      grads = tape.gradient(loss_value, model.trainable_weights)
      optimizer.apply_gradients(zip(grads, model.trainable_weights))
      avg_loss.append(loss_value)
      if(step == step_):
        break
  save_path = manager.save(checkpoint_number=epoch)
  print("Average Training loss --- epoch %d: %.4f"% (epoch, float(sum(avg_loss)/len(avg_loss))))

In [ ]:
model.save("model_training_steps")
model.save("model_training_steps_h5.h5")